In [ ]:
!pip install transformers

In [ ]:
!pip install langchain faiss-cpu pypdf2  python-dotenv

In [ ]:
!pip install langchain_community

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain import hub

In [ ]:
from langchain.schema import Document
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter

pdf_path = ""
reader = PdfReader(pdf_path)
documents = [page.extract_text() for page in reader.pages if page.extract_text()]
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30, separator="\n")
split_texts = text_splitter.split_text("\n".join(documents))
split_documents = [Document(page_content=text) for text in split_texts]

In [ ]:
!pip install langchain_huggingface

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

vectorstore = FAISS.from_documents(split_documents, embeddings)
vectorstore.save_local("faiss_index")
new_vectorstore = FAISS.load_local(
    "faiss_index", embeddings, allow_dangerous_deserialization=True
)

In [ ]:
!pip install langchain_groq

In [ ]:
import os
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from langchain import hub
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
os.environ["GROQ_API_KEY"] = ""

retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
retriever = new_vectorstore.as_retriever()
llm = ChatGroq(model_name="mixtral-8x7b-32768")


combine_docs_chain = create_stuff_documents_chain(
    prompt=retrieval_qa_chat_prompt,
    llm=llm
)

retrieval_chain = create_retrieval_chain(
    retriever, combine_docs_chain
)

In [ ]:
res = retrieval_chain.invoke({"input": "What was the subject of the visit"})
print(res["answer"])

The subject of the visit was a routine nail trim (POD-Nail trim) for the patient, Antonio Figueroa.
